In [1]:
import dotenv
dotenv.load_dotenv()

In [5]:
import os
brave_api_key = os.getenv('BRAVE_API_KEY')
len(brave_api_key)

31

In [6]:
url = "https://api.search.brave.com/res/v1/web/search"

headers = {
    "X-Subscription-Token": brave_api_key,
    "Accept": "application/json",
    "Accept-Encoding": "gzip",
}

params = {
    "q": "Pydantic AI framework python",
    "count": 20,
}

In [7]:
import requests

In [8]:
response = requests.get(url, params=params, headers=headers)
response.raise_for_status()
brave_response = response.json()

In [15]:
brave_response.keys()

dict_keys(['type', 'query', 'mixed', 'videos', 'web'])

In [18]:
search_results = brave_response['web']['results']

for r in search_results:
    print(r['title'])
    print(r['url'])
    print(r['description'])
    print()

Pydantic AI - Pydantic AI
https://ai.pydantic.dev/
Pydantic AI is <strong>a Python agent framework designed to help you quickly, confidently, and painlessly build production grade applications and workflows with Generative AI</strong>.

GitHub - pydantic/pydantic-ai: GenAI Agent Framework, the Pydantic way
https://github.com/pydantic/pydantic-ai
Yet despite virtually every Python ... gave us the same feeling. We built Pydantic AI with one simple aim: <strong>to bring that FastAPI feeling to GenAI app and agent</strong> ......

Pydantic AI: Agent Framework. PydanticAI is a Python Agent Framework… | by Bhavik Jikadara | AI Agent Insider | Medium
https://medium.com/ai-agent-insider/pydantic-ai-agent-framework-02b138e8db71
Pydantic AI is a Python framework that <strong>acts as a bridge between developers and LLMs</strong>, providing tools to create agents — entities that execute specific tasks based on system prompts, functions, and structured outputs.

Overview - Pydantic AI
https://ai.py

In [33]:
from time import sleep

In [34]:
from typing import List, Dict
import requests

def brave_search(query: str, num_results: int = 20) -> List[Dict[str, str]]:
    """
    Search the web

    Args:
        query: The search query string.

    Raises:
        requests.HTTPError: If the API request fails (non-200 response).
        KeyError: If the response format is unexpected.
        requests.RequestException: For network-related errors.
    """

    sleep(1)

    url = "https://api.search.brave.com/res/v1/web/search"

    headers = {
        "X-Subscription-Token": brave_api_key,
        "Accept": "application/json",
        "Accept-Encoding": "gzip",
    }
    
    params = {
        "q": query,
        "count": num_results,
    }

    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()
    brave_response = response.json()

    search_results = brave_response['web']['results']

    output: List[Dict[str, str]] = []
    for r in search_results:
        output.append({
            'title': r['title'],
            'url': r['url'],
            'description': r['description'],
        })

    return output


In [24]:
search_results = brave_search('RAG best practices 2026') 

for r in search_results[:3]:
    print(r['title'])
    print(r['url'])
    print(r['description'])
    print()

RAG in 2026: How Retrieval-Augmented Generation Works for Enterprise ...
https://www.techment.com/blogs/rag-models-2026-enterprise-ai/
Key best practices: ... Pro tip: <strong>Maintain a single source of truth for all RAG-ready content</strong>. ... Use high-precision embeddings tailored for enterprise data — such as domain-tuned embedding models.

Chapter 5: Best Practices for RAG | by Marc Haraoui | Medium
https://medium.com/@marcharaoui/chapter-5-best-practices-for-rag-7770fce8ac81
Some frameworks combine them, like the TRIAD framework highlights Context Relevance (retrieval quality), Faithfulness, and Answer Relevance as the three pillars of RAG evaluation. Example: In practice, you might report something like: “For our test set, Answer Relevance = 0.85, Faithfulness = 0.92, Fluency = 5/5 (human-rated)”, etc., to get a holistic picture.

Building Production RAG Systems in 2026: Complete Architecture Guide | Likhon's Gen AI Blog
https://brlikhon.engineer/blog/building-production-rag

In [26]:
from pydantic_ai import Agent

In [27]:
search_instructions = """
You're a web research assistant. When the user asks a question,
search the web to find the answer. Make 1-2 searches with different
queries to get good coverage. Provide a clear answer based on what
you found.
""".strip()

In [29]:
search_agent = Agent(
    name='search',
    model='gpt-4o-mini',
    instructions=search_instructions,
    tools=[brave_search],
)

In [37]:
from pydantic_ai.messages import FunctionToolCallEvent

class NamedCallback:

    def __init__(self, agent):
        self.agent_name = agent.name

    async def print_function_calls(self, ctx, event):
        # Detect nested streams
        if hasattr(event, "__aiter__"):
            async for sub in event:
                await self.print_function_calls(ctx, sub)
            return

        if isinstance(event, FunctionToolCallEvent):
            tool_name = event.part.tool_name
            args = event.part.args
            print(f"TOOL CALL ({self.agent_name}): {tool_name}({args})")

    async def __call__(self, ctx, event):
        return await self.print_function_calls(ctx, event)

In [40]:
callback = NamedCallback(search_agent)

with search_agent.sequential_tool_calls():
    result = await search_agent.run(
        'AI trends 2026',
        event_stream_handler=callback
    ) 

TOOL CALL (search): brave_search({"query": "AI trends 2026"})
TOOL CALL (search): brave_search({"query": "future AI developments 2026"})


In [48]:
for m in result.all_messages():
    print(m.kind)
    for p in m.parts:
        if p.part_kind == 'user-prompt':
            print(f'  {p.part_kind}: {p.content}')
        elif p.part_kind == 'tool-call':
            print(f'  {p.part_kind}: {p.tool_name} {p.args}')
        elif p.part_kind == 'tool-return':
            print(f'  {p.part_kind}: {p.tool_name}')
        elif p.part_kind == 'text':
            print(f'  {p.part_kind}: {p.content[:100]}...')
    print()

request
  user-prompt: AI trends 2026

response
  tool-call: brave_search {"query": "AI trends 2026"}
  tool-call: brave_search {"query": "future AI developments 2026"}

request
  tool-return: brave_search
  tool-return: brave_search

response
  text: As we look toward 2026, several key trends in artificial intelligence (AI) are anticipated to shape ...



In [51]:
def fetch_content(url: str) -> str:
    """
    Retrieve webpage content.

    Args:
        url: The target webpage URL.

    Returns:
        The textual content of the page as a UTF-8 decoded string.

    Raises:
        requests.HTTPError: If the HTTP request returns a non-success status code.
        requests.RequestException: For network-related errors.
        UnicodeDecodeError: If the response body cannot be decoded as UTF-8.
    """
    jina_prefix = "https://r.jina.ai"
    final_url = f"{jina_prefix}/{url}"

    response = requests.get(final_url)
    response.raise_for_status()

    return response.content.decode("utf-8")


In [52]:
page = fetch_content('https://brlikhon.engineer/blog/building-production-rag-systems-in-2026-complete-architecture-guide')

In [53]:
print(page[:200])

Title: Building Production RAG Systems in 2026: Complete Architecture Guide

URL Source: https://brlikhon.engineer/blog/building-production-rag-systems-in-2026-complete-architecture-guide

Published T


In [57]:
research_instructions = """
You're a web research assistant. When the user asks a question,
search the web to find the answer. Make 1-2 searches with different
queries to get good coverage. Provide a clear answer based on what
you found.

When answering questions:
1. Search for relevant pages
2. Pick the most promising result and fetch its full content
3. Provide a clear answer based on what you found, with source URLs
""".strip()


research_agent = Agent(
    name='research',
    model='gpt-4o-mini',
    instructions=research_instructions,
    tools=[brave_search, fetch_content],
)

In [58]:
callback = NamedCallback(research_agent)

with research_agent.sequential_tool_calls():
    result = await research_agent.run(
        'AI trends 2026',
        event_stream_handler=callback
    ) 

TOOL CALL (research): brave_search({"query": "AI trends 2026"})
TOOL CALL (research): brave_search({"query": "future of AI 2026 predictions"})
TOOL CALL (research): fetch_content({"url": "https://www.ibm.com/think/news/ai-tech-trends-predictions-2026"})
TOOL CALL (research): fetch_content({"url": "https://www.freejobalert.com/article/top-6-ai-trends-shaping-the-future-2026-27764"})


In [59]:
print(result.output)

As we look ahead to 2026, several significant trends in artificial intelligence (AI) are anticipated to shape the technology landscape. Here are some key insights based on recent research and expert predictions:

1. **Autonomous AI Agents**: The future will see a rise of agentic AI systems that can set goals and manage tasks autonomously. These agents will not only assist with routines but will actively drive business operations, making decisions and executing plans across various functions without human intervention (source: [FreeJobAlert](https://www.freejobalert.com/article/top-6-ai-trends-shaping-the-future-2026-27764)).

2. **Multimodal AI**: By 2026, AI systems are expected to integrate various forms of media—text, image, audio, and video—allowing for more intuitive interactions. This capability will enable applications such as generating marketing content tailored to specific demographics just from a photo (source: [FreeJobAlert](https://www.freejobalert.com/article/top-6-ai-tre